In [1]:
import os
import PIL
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
os.getcwd()

'C:\\Users\\HarshGupta\\ml\\sem 6 projects\\LipNet\\codes'

In [3]:
os.chdir('..')

In [4]:
os.chdir('Dataset')
os.chdir('dev')
train = []
label = []
for f in os.listdir():
    os.chdir(f)
    for p in os.listdir():
        img = plt.imread(p)
        label.append(f)
        train.append(img)
    os.chdir('..')
    os.chdir('..')
    os.chdir('Train')
    os.chdir(f)
    for p in os.listdir():
        img = plt.imread(p)
        label.append(f)
        train.append(img)
    os.chdir('..')
    os.chdir('..')
    os.chdir('dev')

In [5]:
train = np.array(train)
label = np.array(label)

In [6]:
from keras.models import Sequential # To initialise the nn as a sequence of layers
from keras.layers import Convolution2D # To make the convolution layer for 2D images
from keras.layers import MaxPooling2D # 
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import CSVLogger
from keras.optimizers import RMSprop
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [7]:
csv=CSVLogger("train_dev_combine.log")
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32,(2,2),input_shape = (224,224,1),strides=2,name='convo1'))
classifier.add(Convolution2D(64,(3,3), activation = 'relu',name='convo2'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 1 - Convolution
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo3'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(BatchNormalization())
classifier.add(GlobalAveragePooling2D())
# As our model is still facing the problem so, we need to increase the regulization
classifier.add(Dropout((0.5)))
classifier.add(Dense(20, activation = 'softmax'))

In [8]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [9]:
from sklearn.cross_validation import train_test_split

C:\Users\HarshGupta\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
X_train,X_test,y_train,y_test = train_test_split(train,label,test_size = 0.25)

In [11]:
X_train.shape

(2100, 224, 224)

In [12]:
y_train.shape

(2100,)

- ## Scaling the data

In [13]:
from keras import backend as K

In [14]:
img_rows = 224
img_cols = 224

In [15]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [16]:
input_shape

(224, 224, 1)

In [17]:
X_train.shape

(2100, 224, 224)

In [18]:
X_train_s = X_train.reshape((X_train.shape[0],) + input_shape)

In [19]:
X_test_s = X_test.reshape((X_test.shape[0],) + input_shape)
X_train_s = X_train_s.astype('float32')
X_test_s = X_test_s.astype('float32')

In [20]:
X_train_s.shape,X_test_s.shape

((2100, 224, 224, 1), (700, 224, 224, 1))

In [21]:
X_train_s /= 255
X_test_s /= 255

In [22]:
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (2100, 224, 224)
2100 train samples
700 test samples


In [23]:
import keras 

In [24]:
y_train = keras.utils.to_categorical(y_train, 20)
y_test = keras.utils.to_categorical(y_test, 20)

In [25]:
y_train.shape

(2100, 20)

In [26]:
y_test.shape

(700, 20)

In [27]:
X_test.shape

(700, 224, 224)

In [28]:
classifier.fit(X_train_s,y_train,batch_size=16,epochs=15,validation_data=(X_test_s, y_test))

Train on 2100 samples, validate on 700 samples
Epoch 1/15
2100/2100 [==============================] - 306s 146ms/step - loss: 3.0043 - acc: 0.0552 - val_loss: 3.3583 - val_acc: 0.0429
Epoch 2/15
2100/2100 [==============================] - 315s 150ms/step - loss: 2.9981 - acc: 0.0571 - val_loss: 3.0781 - val_acc: 0.0629
Epoch 3/15
2100/2100 [==============================] - 325s 155ms/step - loss: 2.9729 - acc: 0.0795 - val_loss: 3.0892 - val_acc: 0.0500
Epoch 4/15
2100/2100 [==============================] - 291s 139ms/step - loss: 2.9343 - acc: 0.0871 - val_loss: 2.9548 - val_acc: 0.0757
Epoch 5/15
2100/2100 [==============================] - 300s 143ms/step - loss: 2.9071 - acc: 0.0862 - val_loss: 2.9207 - val_acc: 0.0786
Epoch 6/15
2100/2100 [==============================] - 294s 140ms/step - loss: 2.8960 - acc: 0.1019 - val_loss: 2.9698 - val_acc: 0.1014
Epoch 7/15
2100/2100 [==============================] - 289s 138ms/step - loss: 2.8718 - acc: 0.1105 - val_loss: 2.9529 - val